# Automated ML

*TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.*

In [1]:
import train_xgb # The module for loading external data
import os
import pandas as pd
import json
import ast
import pickle
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset, Experiment, Model
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

quick-starts-ws-155153
aml-quickstarts-155153
southcentralus
81cefad3-d2c9-4f77-a466-99a7f541c7bb


In [3]:
# Create compute cluster
# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           min_nodes=1,
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
*TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.*

This project processes a data set describing the sale of individual residential property in Ames, Iowa from 2006 to 2010. The data set contains 2930 observations and a large number of explanatory variables (23 nominal, 23 ordinal, 14 discrete, and 20 continuous) involved in assessing home values. The **Ames Housing dataset** was compiled by Dean De Cock for use in data science education.

The project's goal is to train and deploy a machine learning model for **prediction of the sales price based on parameters of a house**.

*TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.*

The Ames Housing dataset is downloaded from Kaggle.com and registered to the workspace.

In [4]:
# Try to load the dataset from the workspace. Otherwise, load it from Kaggle
found = False
ds_key = 'Ames-housing-dataset'
ds_desc = 'Ames Housing training data.'

if ds_key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[ds_key]
    print(f'Found registered {ds_key}, use it.')
    
if not found:
    train, test = train_xgb.load_data_clean(source='kaggle')
    print(f"train.shape = {train.shape}, test.shape = {test.shape}")
    # Register the train dataset
    blob = ws.get_default_datastore()
    dataset = TabularDatasetFactory.register_pandas_dataframe(train, blob, name=ds_key, description=ds_desc)

Found registered Ames-housing-dataset, use it.


## AutoML Configuration

*TODO: Explain why you chose the automl settings and cofiguration you used below.*

As there are time constraints for the experiment, the automl setting:
+ sets experiment timeout to 2 hours,
+ enables early stopping,
+ disables the neural networks (this is a default setting). 

Number of cross validation is set to 3 as the datset has more then 1.000 data points. For the sake of cross platform migration the ONNX compatible models are enabled.

In [5]:
# Choose a name for experiment
experiment_name = 'Ames-housing-AutoML'

experiment=Experiment(ws, experiment_name)

In [6]:
# Set parameters for AutoMLConfig
automl_settings = {
    'experiment_timeout_minutes' : 120,
    'n_cross_validations' : 3,
    'enable_early_stopping' : True,
    'iteration_timeout_minutes' : 5,
    'max_concurrent_iterations' : 4,
    'max_cores_per_iteration' : -1,
    'enable_onnx_compatible_models' : True
}

automl_config = AutoMLConfig(
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    compute_target=cpu_cluster,
    training_data=dataset,
    label_column_name='SalePrice',
    **automl_settings)

## Run Details

*OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?*

Prediction of a sale price based on parameters of a house is an example of a **regression** which is a supervised machine learning task. Regression algorithms used in automated ML defines [Regression Class](https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.constants.supportedmodels.regression?view=azure-ml-py).

+ An example of linear models is [**ElasticNet**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet). It is a linear regression model trained with both l1 and l2-norm regularization of the coefficients. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge.
+ An example of tree-based algorithms is [**XGBoostRegressor**](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn). It is an accurate and effective off-the-shelf procedure that supports a number of different loss functions and is developed with both deep consideration in terms of systems optimization and principles in machine learning.
+ An example of a neural network is [**TensorFlowDNNRegressor**](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNRegressor). DNN models are not enabled by default due to their performance requirements.

The models using gradient tree boosting have usually better performance because they combine predictions of several base estimators. For the same reason the result of the automated ML is usually a voting ensemble. The neural networks can do even better but at costs of compute performance requirements.

*TODO: In the cell below, use the `RunDetails` widget to show the different experiments.*

In [7]:
# TODO: Submit your experiment
aml_run = experiment.submit(automl_config)
RunDetails(aml_run).show()
aml_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Ames-housing-AutoML,AutoML_1e771021-d455-495a-a028-2a419ca9feb4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
Ames-housing-AutoML,AutoML_1e771021-d455-495a-a028-2a419ca9feb4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature h

{'runId': 'AutoML_1e771021-d455-495a-a028-2a419ca9feb4',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-20T12:32:07.015585Z',
 'endTimeUtc': '2021-08-20T13:01:19.444131Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Ames-housing-AutoML","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-155153","workspace_name":"quick-starts-ws-155153","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":100

## Best Model

*TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.*

In [8]:
# Get your best run and save the model from that run.
best_run, fitted_model = aml_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.33.0, current version:1.32.0
Package:azureml-core, training version:1.33.0, current version:1.32.0
Package:azureml-dataprep, training version:2.20.1, current version:2.18.0
Package:azureml-dataprep-native, training version:38.0.0, current version:36.0.0
Package:azureml-dataprep-rslex, training version:1.18.0, current version:1.16.1
Package:azureml-dataset-runtime, training version:1.33.0, current version:1.32.0
Package:azureml-defaults, training version:1.33.0, current version:1.32.0
Package:azureml-interpret, training version:1.33.0, current version:1.32.0
Package:azureml-mlflow, training version:1.33.0, current version:1.32.0
Package:azureml-pipeline-core, training version:1.33.0, current version:1.32.0
Package:azureml-responsibleai, training version:1.33.0, current version:1.32.0
Package:azureml-telemetry, training version:1.33.0, current version:1.32.0
Package:azureml-train-automl-client, training version:1.33.0, current version:1.

Run(Experiment: Ames-housing-AutoML,
Id: AutoML_1e771021-d455-495a-a028-2a419ca9feb4_36,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='regression', working_dir='/mnt/batch/ta...
), random_state=0, reg_alpha=1.4583333333333335, reg_lambda=2.3958333333333335, subsample=0.6, tree_method='hist'))], verbose=False)), ('5', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('elasticnet', ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=1, max_iter=1000, normalize=False, positive=False, precompute=False, random_state=None, selection='cyclic', tol=0.0001, warm_

In [9]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'normalized_root_mean_squared_error': 0.03578930551264211,
 'root_mean_squared_log_error': 0.1302258721688295,
 'explained_variance': 0.8944570643792374,
 'normalized_root_mean_squared_log_error': 0.04236084835831457,
 'normalized_median_absolute_error': 0.014509132245016371,
 'root_mean_squared_error': 25771.878899653584,
 'median_absolute_error': 10448.026129636288,
 'spearman_correlation': 0.9562762595312719,
 'mean_absolute_error': 15609.99578903144,
 'mean_absolute_percentage_error': 9.19763905867747,
 'normalized_mean_absolute_error': 0.021677538937691213,
 'r2_score': 0.8942712633731941,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_1e771021-d455-495a-a028-2a419ca9feb4_36/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_1e771021-d455-495a-a028-2a419ca9feb4_36/predicted_true'}

In [10]:
details = best_run.get_details()
# A pattern for extracting structure of the VotingEnsemble model
voting_ensemble = {k:ast.literal_eval(details['properties'][k]) for k in ['ensembled_algorithms', 'ensemble_weights']}
pd.DataFrame(voting_ensemble).sort_values(by='ensemble_weights', ascending=False)

,ensembled_algorithms,ensemble_weights
0,XGBoostRegressor,0.47
1,XGBoostRegressor,0.40
2,ElasticNet,0.13


In [11]:
# Save metrics and details for ex-post examination
os.makedirs('./aml-outputs', exist_ok=True)
with open('aml-outputs/best_aml_metrics.json', 'w') as file:
    json.dump(best_run_metrics, file)
with open('aml-outputs/best_aml_details.txt', 'w') as file:
    file.write(str(details))

In [12]:
# Explore the main model in the ensemble
main_dict = {}
main_dict['preprocessor'] = str(type(fitted_model.steps[1][1].get_params()['estimators'][0][1].get_params()['steps'][0][1]))

In [13]:
main_dict['estimator_type'] = str(type(fitted_model.steps[1][1].get_params()['estimators'][0][1].get_params()['steps'][1][1]))

In [14]:
main_dict['estimator_param'] = fitted_model.steps[1][1].get_params()['estimators'][0][1].get_params()['steps'][1][1].get_params()

In [15]:
# Save details of the main model for ex-post examination
with open('aml-outputs/best_aml_main_model.json', 'w') as file:
    json.dump(main_dict, file)
main_dict

{'preprocessor': "<class 'sklearn.preprocessing._data.MaxAbsScaler'>",
 'estimator_type': "<class 'azureml.automl.runtime.shared.model_wrappers.XGBoostRegressor'>",
 'estimator_param': {'base_score': 0.5,
  'booster': 'gbtree',
  'colsample_bylevel': 1,
  'colsample_bynode': 1,
  'colsample_bytree': 1,
  'gamma': 0,
  'importance_type': 'gain',
  'learning_rate': 0.1,
  'max_delta_step': 0,
  'max_depth': 3,
  'min_child_weight': 1,
  'missing': nan,
  'n_estimators': 100,
  'n_jobs': -1,
  'nthread': None,
  'objective': 'reg:squarederror',
  'random_state': 0,
  'reg_alpha': 0,
  'reg_lambda': 1,
  'scale_pos_weight': 1,
  'seed': None,
  'silent': None,
  'subsample': 1,
  'verbosity': 0,
  'tree_method': 'auto',
  'verbose': -10}}

In [16]:
# Check the path to the model
for i,n in enumerate(best_run.get_file_names()):
    print(i,n)

0 automl_driver.py
1 azureml-logs/55_azureml-execution-tvmps_f910f983a8c5ed8eb8dbf85b5daf9c414008c6e03c795e484c51c33c7862743f_d.txt
2 azureml-logs/65_job_prep-tvmps_f910f983a8c5ed8eb8dbf85b5daf9c414008c6e03c795e484c51c33c7862743f_d.txt
3 azureml-logs/70_driver_log.txt
4 azureml-logs/75_job_post-tvmps_f910f983a8c5ed8eb8dbf85b5daf9c414008c6e03c795e484c51c33c7862743f_d.txt
5 azureml-logs/process_info.json
6 azureml-logs/process_status.json
7 explanation/5b1854a8/eval_data_viz.interpret.json
8 explanation/5b1854a8/expected_values.interpret.json
9 explanation/5b1854a8/features.interpret.json
10 explanation/5b1854a8/global_names/0.interpret.json
11 explanation/5b1854a8/global_rank/0.interpret.json
12 explanation/5b1854a8/global_values/0.interpret.json
13 explanation/5b1854a8/local_importance_values.interpret.json
14 explanation/5b1854a8/rich_metadata.interpret.json
15 explanation/5b1854a8/true_ys_viz.interpret.json
16 explanation/5b1854a8/visualization_dict.interpret.json
17 explanation/5b18

In [17]:
# Save the best model
os.makedirs('./aml-outputs/', exist_ok=True)
for i in range(33,42):
    print(best_run.get_file_names()[i])
    best_run.download_file(best_run.get_file_names()[i], output_file_path='./aml-outputs/')

outputs/conda_env_v_1_0_0.yml
outputs/env_dependencies.json
outputs/internal_cross_validated_models.pkl
outputs/model.onnx
outputs/model.pkl
outputs/model_onnx.json
outputs/pipeline_graph.json
outputs/scoring_file_v_1_0_0.py
outputs/scoring_file_v_2_0_0.py


Remember you have to deploy only one of the two models you trained. Perform the steps in the rest of this notebook only if you wish to deploy this model.

*TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.*

**Even if it is not necessary to deploy the AutoML model, I am testing the solution to the "AciDeploymentFailed" error I encountered in previus experiments.**

In [8]:
with open('aml-outputs/best_aml_metrics.json', 'r') as file:
    best_run_metrics = json.load(file)

In [9]:
# Register the best model
model = Model.register(ws, model_path='aml-outputs/model.pkl', model_name='Ames-Housing-AutoML-Model', tags=best_run_metrics)
print(model.name, model.id, model.version, sep='\t')

Registering model Ames-Housing-AutoML-Model
Ames-Housing-AutoML-Model	Ames-Housing-AutoML-Model:1	1


In [10]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                              memory_gb=1,
                                              tags={"data" : "Kaggle", "method" : "AutoML"},
                                              description="Predict Ames Housing Prices",
                                              auth_enabled=True,
                                              enable_app_insights=True)

In [11]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("project-env")
cd = CondaDependencies('aml-outputs/conda_env_v_1_0_0.yml')
env.python.conda_dependencies = cd
# Register environment to re-use later
env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "project-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [12]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'Ames-Housing-AutoML-Model')

# Using another environment as recommended by Akintunde, an Udacity mentor
myenv = Environment.get(workspace=ws, name="AzureML-AutoML")
inference_config = InferenceConfig(entry_script="aml-outputs/scoring_file_v_1_0_0.py", environment=myenv)

service_name = 'ames-housing-aml-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-20 15:00:58+00:00 Creating Container Registry if not exists.
2021-08-20 15:00:59+00:00 Registering the environment.
2021-08-20 15:01:00+00:00 Use the existing image.
2021-08-20 15:01:00+00:00 Generating deployment configuration.
2021-08-20 15:01:01+00:00 Submitting deployment to compute.
2021-08-20 15:01:04+00:00 Checking the status of deployment ames-housing-aml-f110..
2021-08-20 15:04:45+00:00 Checking the status of inference endpoint ames-housing-aml-f110.
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 743 ms, sys: 34.8 ms, total: 778 ms
Wall time: 3min 53s


In [13]:
## Yes, no it works!!
print(service.get_logs())

2021-08-20T15:04:36,686530900+00:00 - rsyslog/run 
2021-08-20T15:04:36,690225900+00:00 - iot-server/run 
2021-08-20T15:04:36,696348200+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-20T15:04:36,711672100+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_3489174eb648a475780c9959ff366072/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-20T15:04:37,243257500+00:00 - iot-server/finish 1 0
2021-08-20T15:04:37,245978900+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (58)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 86
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-08-20 15:04:40,695 | root | INFO | Starting up app insights client
logging socket was 

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [15]:
# Prepare data for request
_ , test = train_xgb.load_data_clean()
data = {'data': test.head().to_dict(orient='list')}

# Replace the next cell with the code from 'Consume' tab of the endpoint
# and delete 'data = {}' assignment as data is defined in this cell!  

TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here

body = str.encode(json.dumps(data))

url = 'http://c0a484ea-faf1-420a-9655-f2ad4967541c.southcentralus.azurecontainer.io/score'
api_key = 'gUMGn4uSgbtKhpQgMX6YLjaF5K2lb1Tv' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'"{\\"result\\": [121027.45623469575, 161643.33678900695, 180129.95527012326, 186432.5618099261, 192173.95002451804]}"'


TODO: In the cell below, print the logs of the web service and delete the service

In [17]:
print(service.get_logs())
service.delete()
# Delete() is used to deprovision and delete the AmlCompute target. 
cpu_cluster.delete()

2021-08-20T15:04:36,686530900+00:00 - rsyslog/run 
2021-08-20T15:04:36,690225900+00:00 - iot-server/run 
2021-08-20T15:04:36,696348200+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-20T15:04:36,711672100+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_3489174eb648a475780c9959ff366072/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-20T15:04:37,243257500+00:00 - iot-server/finish 1 0
2021-08-20T15:04:37,245978900+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (58)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 86
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-08-20 15:04:40,695 | root | INFO | Starting up app insights client
logging socket was 